In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamW
import tensorflow as tf
import pandas as pd

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
train_df = pd.read_csv('train.csv')

In [5]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, max_length=128)

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_df['target'].values  
))
train_dataset = train_dataset.shuffle(len(train_df)).batch(16)

In [11]:
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 3  #keep n mind it takes time :)
optimizer, _ = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=0
)

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [15]:
model.fit(train_dataset, epochs=3) #you can adjust this but it won't be so accurate anymore

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
476/476 [==============================] - 9801s 20s/step - loss: 0.4278 - accuracy: 0.8166
Epoch 2/3
476/476 [==============================] - 69718s 147s/step - loss: 0.3146 - accuracy: 0.8824
Epoch 3/3
476/476 [==============================] - 101234s 213s/step - loss: 0.2229 - accuracy: 0.9199


In [20]:
test_df = pd.read_csv('test.csv')

test_encodings = tokenizer(list(test_df['text']), truncation=True, padding=True, max_length=128)

test_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings))
test_dataset = test_dataset.batch(16)

predictions = model.predict(test_dataset)

predicted_labels = tf.argmax(predictions.logits, axis=-1).numpy()

submission = pd.DataFrame({
    "id": test_df["id"],  # Zakładamy, że kolumna 'id' istnieje
    "target": predicted_labels
})
submission.to_csv("submission_3.csv", index=False)

204/204 [==============================] - 632s 3s/step
